In [2]:
import os
import cv2
import time
import random
import shutil
import numpy as np
import matplotlib.pyplot as plt

from keras.models import model_from_json
#from utils_gen import gen_paths_img_dm, gen_var_from_paths
#from utils_imgproc import norm_by_imagenet
%matplotlib inline
plt.ioff()
import glob




import cv2
import numpy as n
# outside of the current session

In [3]:
import os
import cv2
import numpy as np
#from skimage.measure import compare_psnr, compare_ssim
import matplotlib.pyplot as plt


def callbacks_during_train(model, dis_x, dis_y, dis_path, net, epoch):
    if not os.path.exists('tmp'):
        os.mkdir('tmp')
    # show prediction
    pred = np.squeeze(model.predict(np.expand_dims(dis_x, axis=0)))
    _, (ax_x_ori, ax_y, ax_pred) = plt.subplots(1, 3, figsize=(20, 4))
    ax_x_ori.imshow(cv2.cvtColor(cv2.imread(dis_path), cv2.COLOR_BGR2RGB))
    ax_x_ori.set_title('Original Image')
    ax_y.imshow(np.squeeze(dis_y), cmap=plt.cm.jet)
    ax_y.set_title('Ground_truth: ' + str(np.sum(dis_y)))
    ax_pred.imshow(pred, cmap=plt.cm.jet)
    ax_pred.set_title('Prediction: ' + str(np.sum(pred)))
    plt.savefig('tmp/{}_{}-epoch.png'.format(net, epoch))
    return None


def eval_loss(model, x, y, quality=False):
    preds, DM, GT = [], [], []
    losses_SFN, losses_MAE, losses_MAPE, losses_RMSE = [], [], [], []
    for idx_pd in range(x.shape[0]):
        pred = model.predict(np.array([x[idx_pd]]))
        preds.append(np.squeeze(pred))
        DM.append(np.squeeze(np.array([y[idx_pd]])))
        GT.append(round(np.sum(np.array([y[idx_pd]]))))    # To make sure the GT is an integral value
    for idx_pd in range(len(preds)):
        losses_SFN.append(np.mean(np.square(preds[idx_pd] - DM[idx_pd])))     # mean of Frobenius norm
        losses_MAE.append(np.abs(np.sum(preds[idx_pd]) - GT[idx_pd]))
        losses_MAPE.append(np.abs(np.sum(preds[idx_pd]) - GT[idx_pd]) / GT[idx_pd])
        losses_RMSE.append(np.square(np.sum(preds[idx_pd]) - GT[idx_pd]))

    loss_SFN = np.sum(losses_SFN)
    loss_MAE = np.mean(losses_MAE)
    loss_MAPE = np.mean(losses_MAPE)
    loss_RMSE = np.sqrt(np.mean(losses_RMSE))
    if quality:

        psnr, ssim = [], []
        for idx_pd in range(len(preds)):
            data_range = max([np.max(preds[idx_pd]), np.max(DM[idx_pd])]) - min([np.min(preds[idx_pd]), np.min(DM[idx_pd])])
            psnr_ = compare_psnr(preds[idx_pd], DM[idx_pd], data_range=data_range)
            ssim_ = compare_ssim(preds[idx_pd], DM[idx_pd], data_range=data_range)
            psnr.append(psnr_)
            ssim.append(ssim_)
        return loss_MAE, loss_RMSE, loss_SFN, loss_MAPE, np.mean(psnr), np.mean(ssim)
    print("MAE="+str(loss_MAE))
    print("RMSE="+str(loss_RMSE))
    return loss_MAE, loss_RMSE, loss_SFN, loss_MAPE

net = 'Crowd'
dataset = "B"

In [4]:
import numpy as np
train_x=np.load('../input/federatedlearning/TrainSTBXIP.npy')
train_y=np.load('../input/federatedlearning/TrainSTBYIP.npy')
test_x=np.load('../input/federatedlearning/TestSTBx.npy')
test_y=np.load('../input/federatedlearning/TestSTBy.npy')

In [5]:
length = len(train_x)
t_index = length//3
#Floor division rounds down


client1x = train_x[:t_index]
client2x = train_x[t_index:2*t_index]

client3x = train_x[2*t_index+1:]

In [6]:
client2x.shape

(133, 1030, 770, 3)

In [7]:
length = len(train_y)
t_index = length//3

client1y = train_y[:t_index]
client2y = train_y[t_index:2*t_index]

client3y = train_y[2*t_index+1:]

In [8]:
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Conv2D, Input, MaxPooling2D, concatenate, add, UpSampling2D
from keras.models import Model
from keras.initializers import RandomNormal


from tensorflow.keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.recurrent import SimpleRNN, LSTM, GRU
from keras.layers.wrappers import TimeDistributed

def stbflmodel(input_shape=(None, None, 3)):

    input_flow = Input(shape=input_shape)
    dilated_conv_kernel_initializer = RandomNormal(stddev=0.01)

    # front-end
    x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')(input_flow)
    x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x = Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x = Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x = Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x = Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x = Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)

    # back-end
    x = Conv2D(512, (3, 3), strides=(1, 1), padding='same', dilation_rate=2, activation='relu', kernel_initializer=dilated_conv_kernel_initializer)(x)
    x = Conv2D(512, (3, 3), strides=(1, 1), padding='same', dilation_rate=2, activation='relu', kernel_initializer=dilated_conv_kernel_initializer)(x)
    x = Conv2D(512, (3, 3), strides=(1, 1), padding='same', dilation_rate=2, activation='relu', kernel_initializer=dilated_conv_kernel_initializer)(x)
    x = Conv2D(256, (3, 3), strides=(1, 1), padding='same', dilation_rate=2, activation='relu', kernel_initializer=dilated_conv_kernel_initializer)(x)
    x = Conv2D(128, (3, 3), strides=(1, 1), padding='same', dilation_rate=2, activation='relu', kernel_initializer=dilated_conv_kernel_initializer)(x)
    x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', dilation_rate=2, activation='relu', kernel_initializer=dilated_conv_kernel_initializer)(x)

    output_flow = Conv2D(1, 1, strides=(1, 1), padding='same', activation='relu', kernel_initializer=dilated_conv_kernel_initializer)(x)
    model = Model(inputs=input_flow, outputs=output_flow)

    front_end = VGG16(weights='imagenet', include_top=False)

    weights_front_end = []
    for layer in front_end.layers:
        if 'conv' in layer.name:
            weights_front_end.append(layer.get_weights())
    counter_conv = 0
    for i in range(len(front_end.layers)):
        if counter_conv >= 10:
            break
        if 'conv' in model.layers[i].name:
            model.layers[i].set_weights(weights_front_end[counter_conv])
            counter_conv += 1

    return model

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
LOSS = 'MSE'
optimizer = Adam(learning_rate=1e-5)
model = stbflmodel(input_shape=(None,None, 3))
LOSS = 'MSE'
metrics = ['accuracy']
#optimizer = Adam(learning_rate=1e-5)
#model = stbflmodel(input_shape=(None,None, 3))

#model.compile(optimizer=optimizer, loss='MSE')
model.compile(optimizer=optimizer, loss='MSE',metrics=metrics)
model.summary()
if not os.path.exists('models'):
    os.makedirs('models')
plot_model(model, 'models/{}.png'.format(net))
with open('./models/{}.json'.format(net), 'w') as fout:
    fout.write(model.to_json())

2022-01-29 23:38:11.685607: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-29 23:38:11.800571: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-29 23:38:11.801280: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-29 23:38:11.802624: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

58900480/58889256 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
conv2d (Conv2D)              (None, None, None, 64)    1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, None, None, 64)    36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, None, None, 64)    0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 128)   73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, None, None, 128)   147584    
______________________________________________________________

In [9]:
def weight_scaling_factor():
    
    global_count = 400
    
    local_count = 133
    
    return local_count/global_count


def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final



def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
        
    return avg_grad


def test_model(X_test, Y_test,  model, comm_round):
    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    #logits = model.predict(X_test, batch_size=100)
    logits = model.predict(X_test)
    loss = cce(Y_test, logits)
    acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss

In [10]:
LOSS = 'MSE'
metrics = ['accuracy']
optimizer = Adam(learning_rate=1e-5)
global_model = stbflmodel(input_shape=(None,None, 3))

global_model.compile(optimizer=optimizer, loss='MSE')

In [11]:
import warnings

warnings.filterwarnings('ignore')
import tensorflow as tf

In [12]:
for i in range(100):
     print("comm_round"+str(i))
     global_weights = global_model.get_weights()
     scaled_local_weight_list = list()

     print("enter client1")
        
     LOSS = 'MSE'
     metrics = ['accuracy']
     optimizer = Adam(learning_rate=1e-5)
     local_model = stbflmodel(input_shape=(None,None, 3))

     local_model.compile(optimizer=optimizer, loss='MSE')
        
     local_model.set_weights(global_weights)
     for ep in range(1):
        for idx_train in range(0, 133, 16):
        
           x, y = client1x[idx_train:idx_train+16], client1y[idx_train:idx_train+16]
       
           local_model.fit(x, y, batch_size=1, verbose=1)
        
       
        
     scaling_factor = weight_scaling_factor()
     scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
     scaled_local_weight_list.append(scaled_weights)        
     print("enter client2")
        
     LOSS = 'MSE'
     metrics = ['accuracy']
     optimizer = Adam(learning_rate=1e-5)
     local_model2 = stbflmodel(input_shape=(None,None, 3))

     local_model2.compile(optimizer=optimizer, loss='MSE')
        #set local model weight to the weight of the global model
     local_model2.set_weights(global_weights)
     for ep in range(1):
        for idx_train in range(0, 133, 16):
        
           x, y = client2x[idx_train:idx_train+16], client2y[idx_train:idx_train+16]
    
           local_model2.fit(x, y, batch_size=1, verbose=1)
        
     scaling_factor = weight_scaling_factor()
     scaled_weights = scale_model_weights(local_model2.get_weights(), scaling_factor)
     scaled_local_weight_list.append(scaled_weights)  
     #average_weights = sum_scaled_weights(scaled_local_weight_list)
     
     print("enter client3")
        
     LOSS = 'MSE'
     metrics = ['accuracy']
     optimizer = Adam(learning_rate=1e-5)
     local_model3 = stbflmodel(input_shape=(None,None, 3))

     local_model3.compile(optimizer=optimizer, loss='MSE')
        
     local_model3.set_weights(global_weights)
     for ep in range(1):
        for idx_train in range(0, 133, 16):
        
           x, y = client3x[idx_train:idx_train+16], client3y[idx_train:idx_train+16]
       
           local_model3.fit(x, y, batch_size=1, verbose=1)
        
       
        
     scaling_factor = weight_scaling_factor()
     scaled_weights = scale_model_weights(local_model3.get_weights(), scaling_factor)
     scaled_local_weight_list.append(scaled_weights)
     average_weights = sum_scaled_weights(scaled_local_weight_list)
    #update global model 
     global_model.set_weights(average_weights)
     print("global test")
     
     gl= eval_loss(global_model, test_x, test_y, quality=False)

comm_round0
enter client1


2022-01-29 23:38:31.063559: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 152275200 exceeds 10% of free system memory.
2022-01-29 23:38:31.300333: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 152275200 exceeds 10% of free system memory.
2022-01-29 23:38:31.449442: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-01-29 23:38:33.372457: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


16/16 [==============================] - 13s 169ms/step - loss: 0.0026


2022-01-29 23:38:52.996401: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 152275200 exceeds 10% of free system memory.
2022-01-29 23:38:53.201082: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 152275200 exceeds 10% of free system memory.


16/16 [==============================] - 3s 168ms/step - loss: 0.0025


2022-01-29 23:38:56.051613: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 152275200 exceeds 10% of free system memory.


5/5 [==============================] - 1s 169ms/step - loss: 0.0014
enter client2
5/5 [==============================] - 1s 168ms/step - loss: 4.0330e-04
enter client3
5/5 [==============================] - 1s 168ms/step - loss: 0.0066
global test
MAE=46.152777267407764
RMSE=72.41186850192474
comm_round1
enter client1
5/5 [==============================] - 1s 168ms/step - loss: 0.0012
enter client2
5/5 [==============================] - 1s 168ms/step - loss: 4.1786e-04
enter client3
5/5 [==============================] - 1s 168ms/step - loss: 0.0057
global test
MAE=27.493321623983263
RMSE=40.42230048507475
comm_round2
enter client1
5/5 [==============================] - 1s 168ms/step - loss: 0.0011
enter client2
5/5 [==============================] - 1s 168ms/step - loss: 3.1511e-04
enter client3
5/5 [==============================] - 1s 170ms/step - loss: 0.0053
global test
MAE=29.080397135094753
RMSE=39.75303468935101
comm_round3
enter client1
5/5 [==============================] - 1

In [13]:
global_model.save('fedlmodel3c100.h5')
#globalmodel = tf.keras.models.load_model('../input/weights/fedlmodel.h5')

In [14]:
import numpy as np
import tensorflow as tf
#testx=np.load('../input/federatedlearning/TestSTBx.npy')
#testy=np.load('../input/federatedlearning/TestSTBy.npy')
globalmodel = tf.keras.models.load_model('./fedlmodel3c100.h5')
gl= eval_loss(globalmodel, test_x, test_y, quality=False)

MAE=27.851232649404793
RMSE=32.70727705180186
